# Purpose
This notebook is used to map note_number, velocity, instrument_source, qualities, z to f0_scaled and ld_scaled. It is composed of a LSTM layer, GRU layer and  a dense layer

# Setup google drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 2.x
!pip install -qU ddsp[data_preparation]==1.0.1
!pip install keras-tcn

     |████████████████████████████████| 174kB 5.4MB/s 
     |████████████████████████████████| 215kB 5.7MB/s 
     |████████████████████████████████| 3.8MB 7.2MB/s 
     |████████████████████████████████| 3.6MB 36.8MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████| 9.0MB 5.1MB/s 
     |████████████████████████████████| 5.6MB 21.2MB/s 
     |████████████████████████████████| 20.2MB 1.1MB/s 
     |████████████████████████████████| 378kB 38.1MB/s 
     |████████████████████████████████| 17.7MB 205kB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 2.2MB 41.1MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 112kB 42.9MB/s 
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement dill<0.3.2,>=0.3.1.1, but you'll have dill 0.3

# Make directories to save model and data

In [3]:
import os
import datetime
from tcn import TCN, tcn_full_summary

drive_dir = '/content/drive/My Drive/Sound_generation'
checkpoint_dir = os.path.join(drive_dir, 'mapping/checkpoint')

assert os.path.exists(drive_dir)
print('Drive Directory Exists:', drive_dir)

!mkdir -p "$checkpoint_dir"


Drive Directory Exists: /content/drive/My Drive/Sound_generation


# Download Complete NSynth Guitar Subse

In [4]:
dataset_dir = '/content/complete'
train_dataset_dir = os.path.join(dataset_dir, 'train')
valid_dataset_dir = os.path.join(dataset_dir, 'valid')
test_dataset_dir = os.path.join(dataset_dir, 'test')

train_tfrecord_file = os.path.join(train_dataset_dir, 'complete.tfrecord')
valid_tfrecord_file = os.path.join(valid_dataset_dir, 'complete.tfrecord')
test_tfrecord_file = os.path.join(test_dataset_dir, 'complete.tfrecord')

if not os.path.exists(dataset_dir):
  train = 'https://osr-tsoai.s3.amazonaws.com/complete/train/complete.tfrecord'
  valid = 'https://osr-tsoai.s3.amazonaws.com/complete/valid/complete.tfrecord'
  test = 'https://osr-tsoai.s3.amazonaws.com/complete/test/complete.tfrecord'

  print("Downloading train dataset to {}\n".format(train_dataset_dir))
  !mkdir -p "$train_dataset_dir"
  !curl $train --output $train_tfrecord_file

  print("\nDownloading valid dataset to {}\n".format(valid_dataset_dir))
  !mkdir -p "$valid_dataset_dir"
  !curl $valid --output $valid_tfrecord_file

  print("\nDownloading test dataset to {}\n".format(test_dataset_dir))
  !mkdir -p "$test_dataset_dir"
  !curl $test --output $test_tfrecord_file


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.3G  100 10.3G    0     0  16.2M      0  0:10:53  0:10:53 --:--:-- 16.5M


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  675M  100  675M    0     0  15.3M      0  0:00:43  0:00:43 --:--:-- 15.1M


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211M  100  211M    0     0  13.5M      0  0:00:15  0:00:15 --:--:-- 16.0M


# Copying data to drive or from drive

In [5]:
# !cp -r  /content/drive/MyDrive/Sound_generation/mapping/complete  ./

# Defining Data class provider


In [6]:
import tensorflow as tf
import ddsp.training.data as data

class CompleteTFRecordProvider(data.RecordProvider):
  def __init__(self,
               file_pattern=None,
               example_secs=4,
               sample_rate=16000,
               frame_rate=250,
               map_func=None):
    super().__init__(file_pattern, example_secs, sample_rate,
                      frame_rate, tf.data.TFRecordDataset)
    self._map_func = map_func

  def get_dataset(self, shuffle=True):
    def parse_tfexample(record):
      features = tf.io.parse_single_example(record, self.features_dict)
      if self._map_func is not None:
        return self._map_func(features)
      else:
        return features

    filenames = tf.data.Dataset.list_files(self._file_pattern, shuffle=shuffle)
    dataset = filenames.interleave(
        map_func=self._data_format_map_fn,
        cycle_length=40,
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(parse_tfexample,
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

  @property
  def features_dict(self):
    return {
      'sample_name':
        tf.io.FixedLenFeature([1], dtype=tf.string),
      'note_number':
        tf.io.FixedLenFeature([1], dtype=tf.int64),
      'velocity':
        tf.io.FixedLenFeature([1], dtype=tf.int64),
      'instrument_source':
        tf.io.FixedLenFeature([1], dtype=tf.int64),
      'qualities':
        tf.io.FixedLenFeature([10], dtype=tf.int64),
      'audio':
        tf.io.FixedLenFeature([self._audio_length], dtype=tf.float32),
      'f0_hz':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'f0_confidence':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'loudness_db':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'f0_scaled':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'ld_scaled':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'z':
        tf.io.FixedLenFeature([self._feature_length * 16], dtype=tf.float32),
    }


# Defining feature mapping function


In [27]:
def features_map(features):
  note_number = features['note_number']
  velocity = features['velocity']
  instrument_source = features['instrument_source']
  qualities = features['qualities']
  f0_scaled = features['f0_scaled']
  ld_scaled = features['ld_scaled']
  z = features['z']

  sequence_length = f0_scaled.shape[0]

  def convert_to_sequence(feature):
    channels = feature.shape[0]
    feature = tf.expand_dims(feature, axis=0)

    feature = tf.broadcast_to(feature, shape=(sequence_length, channels))
    feature = tf.cast(feature, dtype=tf.float32)
    
    return feature

  # Normalize data
  # 0-127
  note_number = note_number / 127
  velocity = velocity / 127

  # 0-2
  # 0	acoustic, 1	electronic, 2	synthetic
  instrument_source = instrument_source / 2

  # Prepare dataset for a sequence to sequence mapping
  note_number = convert_to_sequence(note_number)
  velocity = convert_to_sequence(velocity)
  instrument_source = convert_to_sequence(instrument_source)
  qualities = convert_to_sequence(qualities)

  f0_scaled = tf.expand_dims(f0_scaled, axis=-1)
  f0_variation = f0_scaled * 127.0 - tf.cast(note_number, dtype=tf.float32)
  f0_variation = tf.clip_by_value(f0_variation, -1.0, 1.0)
  #f0_variation = tf.expand_dims(f0_variation, axis=-1)

  ld_scaled = tf.expand_dims(ld_scaled, axis=-1)
  z = tf.reshape(z, shape=(sequence_length, 16))

  input = tf.concat(
      [note_number, velocity, instrument_source, qualities, z],
      axis=-1)
  
  output = tf.concat(
      [f0_variation, ld_scaled],
      axis=-1)
  # print(f'f0_variation shape: {f0_variation}' )
  # print(f'f0_scaled shape: {f0_scaled}' )

  return (input, output)


# Create Datasets

In [28]:
batch_size = 16
example_secs = 4
sample_rate = 16000
frame_rate = 250

# Create train dataset
train_data_provider = CompleteTFRecordProvider(
    file_pattern=train_tfrecord_file + '*',
    example_secs=example_secs,
    sample_rate=sample_rate,
    frame_rate=frame_rate,
    map_func=features_map)

train_dataset = train_data_provider.get_batch(
    batch_size,
    shuffle=True,
    repeats=-1)

# Create valid dataset
valid_data_provider = CompleteTFRecordProvider(
    file_pattern=valid_tfrecord_file + '*',
    example_secs=example_secs,
    sample_rate=sample_rate,
    frame_rate=frame_rate,
    map_func=features_map)

valid_dataset = valid_data_provider.get_batch(
    batch_size,
    shuffle=True,
    repeats=-1)

# Create test dataset
test_data_provider = CompleteTFRecordProvider(
    file_pattern=test_tfrecord_file + '*',
    example_secs=example_secs,
    sample_rate=sample_rate,
    frame_rate=frame_rate,
    map_func=features_map)

test_dataset = test_data_provider.get_batch(
    batch_size,
    shuffle=True,
    repeats=-1)


In [18]:
tcn_layer = TCN(input_shape=(1000, 29), return_sequences=True, nb_filters=64, kernel_size=8,nb_stacks=2)
print('Receptive field size =', tcn_layer.receptive_field)


Receptive field size = 1765


# Create and compile mapping model


In [20]:

model = tf.keras.models.Sequential([
                                  tcn_layer,
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.GRU(16, return_sequences=True),
    tf.keras.layers.Dense(2, activation='tanh')
])

loss = tf.keras.losses.MeanSquaredError()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=loss,
    metrics=[tf.keras.losses.MeanSquaredError()])

log_dir = "logs/fit/loudness_f0_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Building the model

In [21]:
x_train, y_train = next(iter(train_dataset))
out = model(x_train)

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tcn_2 (TCN)                  (None, 1000, 64)          771968    
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000, 32)          12416     
_________________________________________________________________
gru_1 (GRU)                  (None, 1000, 16)          2400      
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 2)           34        
Total params: 786,818
Trainable params: 786,818
Non-trainable params: 0
_________________________________________________________________
None


# Load checkpoints

In [23]:
checkpoint_file = os.path.join(checkpoint_dir, 'cp.ckpt')

if os.path.isdir(checkpoint_dir) and os.listdir(checkpoint_dir):
    model.load_weights(checkpoint_file)

# Create training callbacks


In [24]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_file,
    save_weights_only=True,
    verbose=0,
    save_freq='epoch')

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * 0.9

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Train the model


In [25]:
epochs = 100
steps_per_epoch = 100
validation_steps = 10

with tf.device('/device:GPU:0'):
  model.fit(train_dataset,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=[lr_scheduler,tensorboard_callback,checkpoint])

Epoch 1/100
100/100 [==============================] - 42s 352ms/step - loss: 0.1802 - mean_squared_error: 0.1802 - val_loss: 0.1722 - val_mean_squared_error: 0.1722
Epoch 2/100
100/100 [==============================] - 32s 322ms/step - loss: 0.1759 - mean_squared_error: 0.1759 - val_loss: 0.1708 - val_mean_squared_error: 0.1708
Epoch 3/100
100/100 [==============================] - 32s 322ms/step - loss: 0.1762 - mean_squared_error: 0.1762 - val_loss: 0.1698 - val_mean_squared_error: 0.1698
Epoch 4/100
100/100 [==============================] - 32s 322ms/step - loss: 0.1742 - mean_squared_error: 0.1742 - val_loss: 0.1689 - val_mean_squared_error: 0.1689
Epoch 5/100
100/100 [==============================] - 32s 322ms/step - loss: 0.1748 - mean_squared_error: 0.1748 - val_loss: 0.1680 - val_mean_squared_error: 0.1680
Epoch 6/100
100/100 [==============================] - 32s 323ms/step - loss: 0.1728 - mean_squared_error: 0.1728 - val_loss: 0.1673 - val_mean_squared_error: 0.1673
Epoc

# Evaluate the model

In [26]:
model.evaluate(test_dataset,steps=500)

500/500 [==============================] - 61s 121ms/step - loss: 0.1606 - mean_squared_error: 0.1606


[0.16062970459461212, 0.16062970459461212]

In [ ]:
model.save('saved_model')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [ ]:
%load_ext tensorboard
!tensorboard --logdir log_dir 

2021-03-28 09:08:30.496041: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
!tensorboard dev upload \
  --logdir logs/fit\
  --name "Mapping network for loudness and F0" \
  --description "Mapping of note_number, velocity, instrument_source, qualities, z to f0 scaled, and loudness scaled " \
  --one_shot

2021-03-28 09:29:35.802090: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

logs/fit

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&

In [ ]:
!cp -r saved_model 

complete  drive  logs  sample_data  saved_model
